In [1]:
import pandas as pd
import numpy as np

In [2]:
filename = 'kosarak.dat.txt'

In [10]:
def load_data(filename, no_lines = 10000):
    l = []
    count = 0
    with open(filename) as f:
        for line in f:
            l.append(set([int(ele) for ele in line.replace('\n','').split()]))
            count+=1
            if count > no_lines:
                print('Data read into list of sets')
                break
    
    return l
basket_data = load_data(filename)
# basket_data[0]

Data read into list of sets


In [4]:
def create_prod_list(data):
    prod_list = []
    for basket in data:
        for prod in basket:
            if prod not in prod_list:
                prod_list.append([prod])
    print('Product list created!')
    return list(map(frozenset,prod_list))

# c1 = create_prod_list(basket_data)

In [12]:
def create_lk(dataset, ck, threshold=0.01*10000):
    freq_count = {}
    for basket in dataset:
        for pair in ck:
            if pair.issubset(basket):
                if pair in freq_count:
                    freq_count[pair]+=1
                else:
                    freq_count[pair]=1
    
    data_size = len(dataset)
    sort_count = sorted(freq_count.items(), key=lambda x:x[1], reverse = True)
    lk = []
    support_dict = {}
    for ele in sort_count:
        if ele[1] > threshold:
            lk.append(ele[0])
            support_dict[ele[0]] = ele[1]/data_size
    print("Created L set")
    lk.sort()
    return lk, dict(sort_count)
    
# l1, supp_dict = create_lk(basket_data, c1)

In [6]:
# threshold = 0.01*count

# list(l1[5])[:0]

In [20]:
def create_ck(lk,k):
    retList = []
    lenlk = len(lk)
    for i in range(lenlk):
        for j in range(i+1, lenlk): 
            tmp_l1 = list(lk[i])[:k-2]
            tmp_l2 = list(lk[j])[:k-2]
            if tmp_l1==tmp_l2: #if first k-1 elements are equal
                retList.append(lk[i] | lk[j]) #set union
    print("Created C set")
    return retList

In [21]:
def apriori(data, threshold=0.01*10000):
    print("Start Apriori algorithm")
    c1 = create_prod_list(data)
    l1, support_data = create_lk(data, c1)
    L = [l1]
    k = 2
    while (len(L[k-2]) > 0):
        print("Value for K:"+str(k))
        ck = create_ck(L[k-2], k)
        Lk, supK = create_lk(data, ck, threshold)#scan DB to get Lk
        support_data.update(supK)
        L.append(Lk)
        k += 1
    return L, support_data

In [ ]:
sets, support_info = apriori(basket_data, threshold=0.001*10000)

Start Apriori algorithm
Product list created!
Created L set
Value for K:2
Created C set
Created L set
Value for K:3
Created C set
Created L set
Value for K:4


In [ ]:
def calculate_conf(itemset, H, support_data, rl, conf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        tmp_conf = support_data[itemset]/support_data[itemset-conseq] #calc confidence
        if tmp_conf >= conf: 
            print (itemset-conseq,'-->',conseq,'conf:',tmp_conf)
            rl.append((itemset-conseq, conseq, tmp_conf))
            prunedH.append(conseq)
    return prunedH

In [ ]:
def rulesFromConseq(itemset, H, support_data, rl, conf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)):    #try further merging
        Hmp1 = create_ck(H, m+1)     #create Hm+1 new candidates
        Hmp1 = calculate_conf(itemset, Hmp1, support_data, rl, conf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(itemset, Hmp1, support_data, rl, conf)

In [ ]:
def generateRules(frequent_itemsets, support_data, conf=0.7):  #supportData is a dict coming from scanD
    rule_list = []
    for i in range(1, len(frequent_itemsets)):#only get the sets with two or more items
        for itemset in frequent_itemsets[i]:
            H1 = [frozenset([item]) for item in itemset]
            if (i > 1):
                rulesFromConseq(itemset, H1, support_data, rule_list, minConf)
            else:
                calculate_conf(itemset, H1, support_data, rule_list, minConf)
    return rule_list

In [36]:
i = 0
for item in ordered_map:
    if item[1]>=threshold:
        i+=1
    else:
        break
ordered_map = ordered_map[:i]

In [37]:
c1 = dict(ordered_map)
# c1

In [38]:
freq_count2 = {}
for item in l:
    tmp = [ele for ele in item if ele in c1]
    tmp.sort()
    if tmp and len(tmp)>1:
        for i in range(len(tmp)-1):
            for j in range(i+1,len(tmp)):
                if tmp[i] in item and tmp[j] in item:
                    if str(tmp[i])+"_"+str(tmp[j]) in freq_count2:
                        freq_count2[str(tmp[i])+"_"+str(tmp[j])]+=1
                    else:
                        freq_count2[str(tmp[i])+"_"+str(tmp[j])] = 1
                        c2
                

In [39]:
c2 = sorted(freq_count2.items(), key=lambda x:x[1], reverse=True)

In [44]:
c2 = [ele for ele in c2 if ele[1]> threshold]
c2 = dict(c2)


In [41]:
freq_count3 = {}
for item in l:
    tmp = [ele for ele in item if ele in c1]
    tmp.sort()
    if tmp and len(tmp)>1:
        for i in range(len(tmp)-1):
            for j in range(i+1,len(tmp)):
                if tmp[i] in item and tmp[j] in item:
                    if str(tmp[i])+"_"+str(tmp[j]) in freq_count2:
                        freq_count2[str(tmp[i])+"_"+str(tmp[j])]+=1
                    else:
                        freq_count2[str(tmp[i])+"_"+str(tmp[j])] = 1

137

54